In [ ]:
!pip install --upgrade frozendict
!pip install experta
!pip install yfinance --upgrade

  Preparing metadata (setup.py) ... done
  Created wheel for frozendict: filename=frozendict-1.2-py3-none-any.whl size=3149 sha256=35c387ea4a02c367bd550595a31a68158784f44d4791643ae27821d953ee7217
  Stored in directory: /root/.cache/pip/wheels/ff/80/86/2d516a3c26397f67adaa2c848879d4a6359d90a60546ce4e03
Successfully built frozendict
  Attempting uninstall: frozendict
    Found existing installation: frozendict 2.4.4
    Uninstalling frozendict-2.4.4:
      Successfully uninstalled frozendict-2.4.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
yfinance 0.2.43 requires frozendict>=2.3.4, but you have frozendict 1.2 which is incompatible.
  Using cached frozendict-2.4.4-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (23 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.6/94.6 kB 1.7 MB/s eta 0:00:00
Using cached frozendict-2.4.4-cp310-cp310-m

In [32]:
from experta import *
import random

# 定義知識庫的事實
class EnergyFact(Fact):
    """表示設備狀態或環境條件"""
    pass

# 根據用電量返回電價的函數
def get_energy_price(usage, is_summer):
    if usage <= 120:
        return 1.68
    elif 121 <= usage <= 330:
        return 2.45 if is_summer else 2.16
    elif 331 <= usage <= 500:
        return 3.70 if is_summer else 3.03
    elif 501 <= usage <= 700:
        return 5.04 if is_summer else 4.14
    elif 701 <= usage <= 1000:
        return 6.24 if is_summer else 5.07
    else:
        return 8.46 if is_summer else 6.63

# 創建專家系統類
class AdvancedEnergySavingExpert(KnowledgeEngine):

    # 冷氣相關規則
    @Rule(EnergyFact(temperature=P(lambda t: t < 15), season='winter'))
    def increase_heating(self):
        print("建議增加供暖以保持舒適")

    @Rule(EnergyFact(temperature=P(lambda t: t > 26), season='summer'))
    def increase_ac(self):
        print("建議調低冷氣溫度來保持舒適，但不應低於26度")

    @Rule(EnergyFact(temperature=P(lambda t: 15 <= t <= 26), time_of_day='night', ac_on=True))
    def turn_off_ac_at_night(self):
        print("夜間溫度舒適，建議關閉冷氣以節省能源")

    # 燈光相關規則
    @Rule(EnergyFact(daylight=True, light_on=True))
    def turn_off_lights(self):
        print("白天光線充足，建議關閉燈光")

    # 家電使用建議 - 根據新的電價閾值更新
    @Rule(EnergyFact(energy_price=P(lambda price: price < 5), appliance="washing_machine"))
    def suggest_use_appliance_low_price(self):
        print("能源價格較低，建議現在使用洗衣機等高能耗設備")

    @Rule(EnergyFact(appliance_age=P(lambda age: age > 10), appliance_efficiency='low'))
    def replace_old_appliance(self):
        print("家電年齡超過10年且能效較低，建議更換為高效能家電")

    # 綜合性推理規則
    @Rule(EnergyFact(temperature=P(lambda t: t > 30), energy_price=P(lambda price: price > 5)))
    def warn_high_cost_high_temp(self):
        print("高溫且能源價格高，建議減少冷氣使用並考慮其他降溫措施")

    @Rule(EnergyFact(temperature=P(lambda t: t < 15), appliance="heating", energy_price=P(lambda price: price > 5)))
    def manage_heating_cost(self):
        print("低溫且能源價格高，建議適當控制供暖溫度以減少成本")

# 創建專家系統引擎
engine = AdvancedEnergySavingExpert()

# 初始化推理引擎
engine.reset()

# 模擬動態事實（溫度、時間、能源價格等）
temperature = random.randint(8, 35)
time_of_day = random.choice(['morning', 'afternoon', 'night'])
daylight = random.choice([True, False])
usage = random.randint(50, 1200)  # 模擬用電量
is_summer = random.choice([True, False])  # 是否是夏季
energy_price = get_energy_price(usage, is_summer)

# 輸出動態情境
season = '夏月' if is_summer else '非夏月'
print(f"當前季節: {season}")
print(f"當前溫度: {temperature}度")
print(f"當前時間: {time_of_day}")
print(f"是否白天: {'是' if daylight else '否'}")
print(f"當前用電量: {usage} 度")
print(f"當前能源價格: {energy_price} 每度")

# 插入事實並運行推理引擎
engine.declare(EnergyFact(temperature=temperature, season='summer' if is_summer else 'winter', time_of_day=time_of_day, ac_on=True, daylight=daylight, light_on=True, energy_price=energy_price, appliance="washing_machine", appliance_age=12, appliance_efficiency='low'))
engine.run()


當前季節: 夏月
當前溫度: 29度
當前時間: morning
是否白天: 是
當前用電量: 394 度
當前能源價格: 3.7 每度
家電年齡超過10年且能效較低，建議更換為高效能家電
白天光線充足，建議關閉燈光
建議調低冷氣溫度來保持舒適，但不應低於26度
能源價格較低，建議現在使用洗衣機等高能耗設備
